In [1]:
import os, sys
import tensorflow as tf
import numpy as np
import cv2
import gc
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
import CNN.vgg16 as vgg16
import CNN.utils as util

# Data Loader and Batch

In [2]:
def dataLoad():
    
    print("Data Load Starting..!")
    path = "../SPORT_QA/DataSet(190408)/"
    path = [os.path.join(path, x) for x in os.listdir(path)] 
    path2 = [os.path.join(x, y) for x in path if os.path.isdir(x)  for y in os.listdir(x)]
    #print(path2)

    path3= [os.path.join(x, y) for x in path2 if x[-12:]=="Ground_Truth"for y in os.listdir(x)]

    path2 = [os.path.join(x, y) for x in path3 if os.path.isdir(x) for y in os.listdir(x)]
    path3 = [os.path.join(x, y) for x in path2 if os.path.isdir(x) for y in os.listdir(x) if y[-3:]=="txt"]
    random.shuffle(path3)
        
    Train_data = path3[:len(path3)*3//4] 
    Test_data = path3[len(path3)*3//4:] 
    
    print("DataLoad Done..")
    print("Train data.. " , len(Train_data))
    print("Test data.. " , len(Test_data))
    return Train_data, Test_data


In [3]:
class Batch:
    
    def __init__(self, data_list):
        self.data_list = data_list
        self.now = 0
        self._shuffle_list()
        
        
    def next_batch(self, batch_size):
        
        self.end = self.now + batch_size
        if self.end > len(self.data_list):
            return None, None
        
        else:
            img, scale = self.image_load_and_batch(self.data_list[self.now:self.end])
            label = self.Find_ball(self.data_list[self.now:self.end], scale)
            self.now = self.end
            return img, label
            
        
    
    def _shuffle_list(self):
        random.shuffle(self.data_list)
        
        
    def image_load_and_batch(self,img_list):
        im = []
        scale = []

        for img_name in img_list:

            #img = cv2.resize(cv2.imread(img_name),(224,224))
            img_name = img_name.replace("Ground_Truth", "Parse_Data").replace(".txt", ".jpg")
            img = cv2.imread(img_name)
            y_, x_, _ = np.shape(img)
            img = cv2.resize(img, (224,224))
            y_scale = 224 / y_
            x_scale = 224 / x_
            scale.append([x_scale, y_scale])
            b, g, r = cv2.split(img)
            im.append(cv2.merge([r,g,b]))

            #print(np.shape(im),np.shape(labels))
        return im, scale
        
    
    def Find_ball(self, data, scale):
        
        label_maps = []
        
        for i,d in enumerate(data):
            f = open(d, 'r')
            
            while True:
                line = f.readline()
                if not line:
                    label_maps.append([112,112])
                    break
                line = line[1:-2].split(",")
                label = str(line[0][1:-1])
                if label == 'sports ball':
                    label_maps.append([(int(np.round(int(line[1][1:])*scale[i][0])) + int(np.round((int(line[1][1:]) + int(line[3][1:]))*scale[i][0])))//2,
                                       (int(np.round(int(line[2][1:])*scale[i][1]))+int(np.round((int(line[2][1:]) + int(line[4][1:]))*scale[i][1])))//2])
                    break
                

            
        return label_maps


    

# Model Definition

In [ ]:
class REG_Model:
    
    def __init__(self,sess, n_class):
        self.sess = sess
        self.n_class = n_class
        self.vgg = vgg16.Vgg16()
        self._build_net()
        
    def _build_net(self):
        learning_rate = 0.0001
        #Declare Input Placeholder
        self.training = tf.placeholder(tf.bool)
        self.keep = tf.placeholder(tf.float32)
        self.X = tf.placeholder(tf.float32, [None, 224,224,3])
        self.Y = tf.placeholder(tf.float32, [None, 2])
        
        #Model Start
        self.vgg.build(self.X)
        
        #vgg.conv5_3,vgg.pool4, vgg.pool3
        layer = self._max_pool(self.vgg.conv5_3, 2, 2)
        layer = tf.layers.flatten(layer)
        
        with tf.variable_scope("Fully_Connected1"):
            layer = self._full_block(layer, 4096,  self.keep, dropout=True)
            
        with tf.variable_scope("Fully_Connected2"):
            layer = self._full_block(layer, 4096,  self.keep, dropout=True)
            
        with tf.variable_scope("Fully_Connected3"):
            layer = tf.layers.dense(layer, self.n_class)
        
        with tf.variable_scope("Fully_Connected4"):
            layer = tf.layers.dense(layer, self.n_class)

            
        
#         ori_image = tf.constant([1,1,224,224], shape = [16,4], dtype=tf.float32)
#         bias = tf.getvariable("b", [4])
        
        
#         dxp = tf.slice(layer, [0,0], [-1,1])
#         dyp = tf.slice(layer, [0,1], [-1,1])
#         dwp = tf.slice(layer, [0,2], [-1,1])
#         dhp = tf.slice(layer, [0,3], [-1,1])
        
#         px = tf.constant(0, shape = [1], dtype=tf.float32)
#         py = tf.constant(0, shape = [1], dtype=tf.float32)
#         pw = tf.constant(224, shape = [1,1], dtype=tf.float32)
#         ph = tf.constant(224, shape = [1,1], dtype=tf.float32)
        
        
#         gx = tf.matmul(dxp , pw) + px
#         gy = tf.matmul(dxp , ph) + py
#         gw = tf.matmul(tf.exp(dwp), pw)
#         gh = tf.matmul(tf.exp(dhp), ph)
        
        
#         tx = tf.divide(tf.subtract(gx, px), pw)
#         ty = tf.divide(tf.subtract(gy, py), ph)
#         tw = tf.log(tf.divide(gw, pw))
#         th = tf.log(tf.divide(gh, ph))
        
        #Last Optimizer    
        self.Logits = layer
#         self.Logits = tf.add(tf.multiply(layer, ori_image),b)
        
#         plxy = tf.slice(self.Logits, [0,0], [-1,2]) 
#         prxy = tf.slice(self.Logits, [0,2], [-1,2])
#         olxy = tf.slice(self.Y, [0,0], [-1,2])
#         orxy =  tf.slice(self.Y, [0,2], [-1,2])
        
        #self.Loss = tf.reduce_mean(tf.add(tf.nn.l2_loss(tf.subtract(plxy,olxy)), tf.nn.l2_loss(tf.subtract(prxy,orxy))))
        self.Loss = tf.reduce_mean(tf.nn.l2_loss(tf.subtract(self.Logits,self.Y)))
        self.Optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.Loss)
        
        #Get Accuracy
        
#         lw = tf.subtract(tf.slice(self.Logits, [0,2], [-1,1]) ,tf.slice(self.Logits, [0,0], [-1,1]))
#         lh = tf.subtract(tf.slice(self.Logits, [0,3], [-1,1]) , tf.slice(self.Logits, [0,1], [-1,1]))
#         ow = tf.subtract(tf.slice(self.Y, [0,2], [-1,1]) , tf.slice(self.Logits, [0,0], [-1,1]))
#         oh = tf.subtract(tf.slice(self.Y, [0,3], [-1,1]) , tf.slice(self.Logits, [0,1], [-1,1]))
        
#         l_area = tf.multiply(lw, lh)
#         o_area = tf.multiply(ow, oh)
        

        
        self.accuracy = tf.reduce_mean(tf.divide(tf.constant(1.),tf.sqrt(tf.square(tf.subtract(self.Logits,self.Y)))))
        
    def _conv_block(self, input_data, filter_, kernel_size_, keep_prob_, dropout=False):
        
        output = tf.layers.conv2d(input_data, filters=filter_, kernel_size=kernel_size_, name = "Convolution", activation=tf.nn.relu,
                                 padding="SAME")
        if dropout==True:
            output = tf.layers.dropout(inputs=output, rate=keep_prob_, training=self.training)
            
        return output
    
    def _full_block(self, input_data, output_, keep_prob_, dropout=False):
        
        output = tf.layers.dense(input_data, units = output_, name = "Full", activation=tf.nn.relu)
        if dropout==True:
            output = tf.layers.dropout(inputs=output, rate=keep_prob_, training=self.training)
            
        return output
    
    def _up_sample(self, input_data, filter_, kernel_size_, stride_):
        up_sample = tf.layers.conv2d_transpose(input_data, filters=filter_, kernel_size= kernel_size_, strides=stride_,
                                   padding='SAME')
        return up_sample
    
    def _max_pool(self, input_data, kernel_size_, stride_):
        return tf.layers.max_pooling2d(inputs = input_data, pool_size=kernel_size_,
                                       strides=stride_, padding="valid",name="maxpooling")
    
    def predict(self, input_data, keep_prob, training=False):
        return self.sess.run(self.Logits, feed_dict={self.X:input_data, self.training:training, self.keep:keep_prob})
    
    def get_accuracy(self, input_data, input_label, keep_prob, training=False):
        return self.sess.run(self.accuracy, feed_dict={self.X:input_data, self.Y:input_label,
                                                       self.training:training, self.keep:keep_prob})
    
    def train(self, input_data, input_label, keep_prob, training=True):
        return self.sess.run([self.Loss, self.Optim, self.accuracy], feed_dict ={self.X:input_data,self.Y:input_label,
                                                                                 self.training:training, self.keep:keep_prob})
        
        

# Train Start

In [ ]:
tf.reset_default_graph()

Train_data, Test_data = dataLoad()



n_class = 2
keep_prob = 0.85
batch_size = 16
n_epochs = 10000


print("Training.. Start..")
with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7 )))) as sess:
    
    Model = REG_Model(sess, n_class=n_class)
    
    sess.run(tf.initialize_all_variables())
    sess.run(tf.local_variables_initializer())
    saver = tf.train.Saver(max_to_keep = 100)

    for epoch in tqdm_notebook(range(n_epochs)):
        
        print("Shuffle Data And Batches it Now epoch : ", epoch)
        train_batch = Batch(Train_data)
        a_loss = 0
        a_acc = 0
        print("Batch.. Done..")

        print("Train Start.. ")
        while True:

            train_img, train_label = train_batch.next_batch(batch_size)
            if train_img == None: break
            #print(len(train_img))
            loss, _, acc = Model.train(train_img, train_label, keep_prob)
            a_loss += loss
            a_acc += acc
        print("Train End..")
        print("Now Mean ACC : ", a_acc/(len(Train_data)// batch_size) *100, "Now ALL Loss : ",
              a_loss, "Now Mean Loss : ", a_loss/(len(Train_data)// batch_size))
        
        print("\n\n Test Start..")
        test_batch = Batch(Test_data)
        a_acc = 0
        
        while True:

            test_img, test_label = test_batch.next_batch(batch_size)
            if test_img == None: break
            acc = Model.get_accuracy(test_img, test_label, keep_prob)
            a_acc += acc
            
        saver.save(sess, './New_Seg_Result/Model/REG_' + str(epoch) + '.ckpt')   
        print("Test Done..")
        print("Test ACC : ", a_acc/(len(Test_data)//batch_size) * 100, "\n\n")
        f = open("./New_Seg_Result/Result_reg.txt", "a")
        f.write(str(epoch)+ " : "+ str(a_acc/(len(Test_data)//batch_size) * 100))
        f.close()

Data Load Starting..!
DataLoad Done..
Train data..  2483
Test data..  828
Training.. Start..
D:\Ball_Detection\CNN\vgg16.npy
npy file loaded
build model started
build model finished: 4s
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Shuffle Data And Batches it Now epoch :  0
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  0.0410438931289865 Now ALL Loss :  1369728067840.0 Now Mean Loss :  8836955276.387096


 Test Start..
Test Done..
Test ACC :  0.6409028083926029 


Shuffle Data And Batches it Now epoch :  1
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  0.05734041056712158 Now ALL Loss :  647293601856.0 Now Mean Loss :  4176087753.9096775


 Test Start..
Test Done..
Test ACC :  0.34052817899660737 


Shuffle Data And Batches it Now epoch :  2
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  0.11993474950413069 Now ALL Loss :  330108777280.0 Now Mean Loss :  2129734046.967742


 Test Start..
Test Done..
Test ACC :  0.5324600962921977 


Shuffle Data And Batches it Now epoch :  3
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  0.11793814877408645 Now ALL Loss :  166120051456.0 Now Mean Loss :  1071742267.4580646


 Test Start..
Test Done..
Test ACC :  1.094056197496898 


Sh

Test Done..
Test ACC :  1.0761044389915233 


Shuffle Data And Batches it Now epoch :  33
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.1432890137357097 Now ALL Loss :  3479379107.3125 Now Mean Loss :  22447607.143951613


 Test Start..
Test Done..
Test ACC :  1.0819875178676024 


Shuffle Data And Batches it Now epoch :  34
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.2659319303929806 Now ALL Loss :  5688934788.9375 Now Mean Loss :  36702805.08991936


 Test Start..
Test Done..
Test ACC :  1.7031919211149216 


Shuffle Data And Batches it Now epoch :  35
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.1841825272647604 Now ALL Loss :  1989253342.859375 Now Mean Loss :  12833892.534576613


 Test Start..
Test Done..
Test ACC :  1.074465082994863 


Shuffle Data And Batches it Now epoch :  36
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.1643628097109255 Now ALL Loss :  1581387507.3125 Now Mean Loss :  10202500.047177419


 Test Star

Now Mean ACC :  1.315392497326097 Now ALL Loss :  231243492.5 Now Mean Loss :  1491893.5


 Test Start..
Test Done..
Test ACC :  1.0455936869131583 


Shuffle Data And Batches it Now epoch :  66
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.2863866984844208 Now ALL Loss :  48801786.171875 Now Mean Loss :  314850.23336693546


 Test Start..
Test Done..
Test ACC :  1.0459059465895681 


Shuffle Data And Batches it Now epoch :  67
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.4459095222334708 Now ALL Loss :  48299211.640625 Now Mean Loss :  311607.8170362903


 Test Start..
Test Done..
Test ACC :  1.0489667108392013 


Shuffle Data And Batches it Now epoch :  68
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.287086525511357 Now ALL Loss :  42771652.46875 Now Mean Loss :  275946.1449596774


 Test Start..
Test Done..
Test ACC :  1.0454966321441472 


Shuffle Data And Batches it Now epoch :  69
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :

Train Start.. 
Train End..
Now Mean ACC :  1.63248433761539 Now ALL Loss :  35557713.765625 Now Mean Loss :  229404.60493951614


 Test Start..
Test Done..
Test ACC :  1.1509425061590532 


Shuffle Data And Batches it Now epoch :  99
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.5457766430993234 Now ALL Loss :  35467353.40625 Now Mean Loss :  228821.63487903227


 Test Start..
Test Done..
Test ACC :  1.142145810174007 


Shuffle Data And Batches it Now epoch :  100
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  2.3238615962045808 Now ALL Loss :  35386402.28125 Now Mean Loss :  228299.36955645162


 Test Start..
Test Done..
Test ACC :  1.1404011562904892 


Shuffle Data And Batches it Now epoch :  101
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1.5775004936562431 Now ALL Loss :  35325372.390625 Now Mean Loss :  227905.6283266129


 Test Start..
Test Done..
Test ACC :  1.1379208799232454 


Shuffle Data And Batches it Now epoch :  102
Batch.. Done..

...

Now Mean ACC :  21.01656899577187 Now ALL Loss :  4058967.005859375 Now Mean Loss :  26186.88390877016


 Test Start..
Test Done..
Test ACC :  22.92426834795989 


Shuffle Data And Batches it Now epoch :  1357
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  27.42448213119661 Now ALL Loss :  4015841.404296875 Now Mean Loss :  25908.65422127016


 Test Start..
Test Done..
Test ACC :  74.88341714237251 


Shuffle Data And Batches it Now epoch :  1358
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  35.10889189618249 Now ALL Loss :  4029088.8828125 Now Mean Loss :  25994.121824596776


 Test Start..
Test Done..
Test ACC :  142.41824452491366 


Shuffle Data And Batches it Now epoch :  1359
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  31.18012880125353 Now ALL Loss :  4044983.9140625 Now Mean Loss :  26096.670413306452


 Test Start..
Test Done..
Test ACC :  256.1387222041102 


Shuffle Data And Batches it Now epoch :  1360
Batch.. Done..
Train Start.. 
Train

Now Mean ACC :  44.67606536082683 Now ALL Loss :  4084291.3857421875 Now Mean Loss :  26350.267004788308


 Test Start..
Test Done..
Test ACC :  30.16803048404993 


Shuffle Data And Batches it Now epoch :  1389
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  1367.867056934103 Now ALL Loss :  4045169.849609375 Now Mean Loss :  26097.86999747984


 Test Start..
Test Done..
Test ACC :  25.78139360628876 


Shuffle Data And Batches it Now epoch :  1390
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  22.39553981009991 Now ALL Loss :  4062874.2431640625 Now Mean Loss :  26212.09189138105


 Test Start..
Test Done..
Test ACC :  151.00520954412573 


Shuffle Data And Batches it Now epoch :  1391
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  39.75762588602881 Now ALL Loss :  4045659.7607421875 Now Mean Loss :  26101.030714465727


 Test Start..
Test Done..
Test ACC :  307.7717316647371 


Shuffle Data And Batches it Now epoch :  1392
Batch.. Done..
Train Start..

Now Mean ACC :  49.204386833213995 Now ALL Loss :  4016819.765625 Now Mean Loss :  25914.96622983871


 Test Start..
Test Done..
Test ACC :  23.316906640926994 


Shuffle Data And Batches it Now epoch :  1421
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  25.043985632158094 Now ALL Loss :  4010651.5517578125 Now Mean Loss :  25875.171301663308


 Test Start..
Test Done..
Test ACC :  58.16948824361259 


Shuffle Data And Batches it Now epoch :  1422
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  37.59493663426369 Now ALL Loss :  4024489.708984375 Now Mean Loss :  25964.449735383063


 Test Start..
Test Done..
Test ACC :  134.74902217002474 


Shuffle Data And Batches it Now epoch :  1423
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  25.34845322130188 Now ALL Loss :  4003591.5048828125 Now Mean Loss :  25829.622612147177


 Test Start..
Test Done..
Test ACC :  154.61364638279466 


Shuffle Data And Batches it Now epoch :  1424
Batch.. Done..
Train Start.

Now Mean ACC :  27.948739824275815 Now ALL Loss :  4041303.513671875 Now Mean Loss :  26072.92589465726


 Test Start..
Test Done..
Test ACC :  36.55244358906559 


Shuffle Data And Batches it Now epoch :  1453
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  19.312800591991795 Now ALL Loss :  4072234.884765625 Now Mean Loss :  26272.48312752016


 Test Start..
Test Done..
Test ACC :  1196.966834059533 


Shuffle Data And Batches it Now epoch :  1454
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  39.10839981129093 Now ALL Loss :  4030086.1123046875 Now Mean Loss :  26000.55556325605


 Test Start..
Test Done..
Test ACC :  32.03696596856211 


Shuffle Data And Batches it Now epoch :  1455
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  83.14914227493347 Now ALL Loss :  4043282.8623046875 Now Mean Loss :  26085.695885836692


 Test Start..
Test Done..
Test ACC :  85.83742917752734 


Shuffle Data And Batches it Now epoch :  1456
Batch.. Done..
Train Start.. 

Now Mean ACC :  26.94797013075121 Now ALL Loss :  4019425.814453125 Now Mean Loss :  25931.77944808468


 Test Start..
Test Done..
Test ACC :  23.296316099517487 


Shuffle Data And Batches it Now epoch :  1485
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  368.3479111954089 Now ALL Loss :  4063975.1884765625 Now Mean Loss :  26219.19476436492


 Test Start..
Test Done..
Test ACC :  142.8045181301879 


Shuffle Data And Batches it Now epoch :  1486
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  27.884675019210388 Now ALL Loss :  4011048.6513671875 Now Mean Loss :  25877.733234627016


 Test Start..
Test Done..
Test ACC :  226.40224755102514 


Shuffle Data And Batches it Now epoch :  1487
Batch.. Done..
Train Start.. 
Train End..
Now Mean ACC :  28.47775547975494 Now ALL Loss :  4042512.3779296875 Now Mean Loss :  26080.72501890121


 Test Start..
Test Done..
Test ACC :  73.49839114090976 


Shuffle Data And Batches it Now epoch :  1488
Batch.. Done..
Train Start.

# DeMo Start

load test data

In [ ]:
# DeMo
WantFolder = "Free_A"
WantLabel = "00010"
WantLabel2 = "0"
Free = "free"
Want_Restore = "REG_21.ckpt"

#Test_Image_Load
file_path=[]
basepath = "../SPORT_QA/DataSet(190408)"
basepath2 = "Parse_Data"

img_path = os.path.join(basepath, WantFolder, basepath2, WantLabel, WantLabel2)

for x in os.listdir(os.path.join("result_img", Free, WantLabel, WantLabel2)):
    if x[-3:]=="txt":
        file_path.append(os.path.join("result_img", Free, WantLabel, WantLabel2, x))
res_path = os.path.join( "New_Seg_Result","Model", Want_Restore)
img_list = sorted([os.path.join(img_path,x ) for x in os.listdir(img_path) if x[-3:]=="jpg"])

In [ ]:
img_list

In [ ]:
tf.reset_default_graph()

num =0
keep_prob = 0.85

print("Training.. Start..")
with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7 )))) as sess:
    
    Model = REG_Model(sess, n_class=n_class)
    
    sess.run(tf.initialize_all_variables())
    sess.run(tf.local_variables_initializer())
    saver = tf.train.Saver(max_to_keep = 8000000)
    
    saver.restore(sess, res_path)
    for img in img_list:
        
        fig, ax = plt.subplots(1, figsize =(16,16))
        ax.axis('off')
        
        
        im = cv2.resize(cv2.imread(img), (224,224))
        b,g,r = cv2.split(im)
        im = cv2.merge((r,g,b))
        
        logit=  Model.predict([im], keep_prob)
        print(logit)
        
        im_ = cv2.rectangle(im, (logit[0][:2]), (logit[0][2:]), (255,0,0), 3) 
        
        ax.imshow(im_.astype(np.uint8))
        fig.savefig("Ball_Find_Result/"+str(num)+".jpg")
        plt.show()
        num +=1
    